## GroupMe

using python to make sense of our GroupMe discussions

## API  
Create Developer account  
Create Application  
 - this is **ReportingViaJupyter**  

https://dev.groupme.com/applications


## Docs

RestAPI:
https://dev.groupme.com/docs/v3

Groupy - Python Wrapper
https://github.com/rhgrant10/Groupy

`pip install GroupyAPI`

In [1]:
import json

from groupy import Client
from groupy import attachments

from datetime import datetime
from dateutil import tz

import os
import collections

In [2]:
# Config

# API Token
myApiToken = '12a551408b7c0136238a3aad6ef5005b'

# Auto-detect zones:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()

In [3]:
client = Client.from_token(myApiToken)

In [4]:
groups = list(client.groups.list_all())

In [11]:
cwd = os.getcwd()
print(cwd)

/Users/Rob/Projects/Notebooks/groupme


In [15]:
# Group Of Interest
searchGroup = '2019 NHL Draft' #'☕️ Friday Coffee' #'💩 Misc Crap'
theGroup = None

for group in groups:
    if group.name == searchGroup:
        theGroup = group
        print('Working on group: ' + theGroup.name)
        
        #Create a working directory for this group
        directory = theGroup.name
        if not os.path.exists(directory):
            os.makedirs(directory)
        print(" - output will be saved in folder of same name")
        #os.chdir(directory)
if theGroup is None:
    print("Not found.  Pick one of these:")
    for group in groups:
        print(group.name)


Working on group: 2019 NHL Draft
 - output will be saved in folder of same name


### Functions

In [16]:
def format_mesage(message):
    message_datetime_utc = message.created_at                                                     
    message_datetime_local = message_datetime_utc.astimezone(to_zone)
    m_datetime = message_datetime_local.strftime('%Y-%m-%d %H:%M:%S')
    
    if message.text is None:
        m_text = '*no message*'
    else:
        m_text = message.text

    m_attachments = '['
    if message.attachments is not None:
        for attachment in message.attachments:
            m_attachments = m_attachments + attachment.type + ' '
    else:
        m_attachments = m_attachments + '-'
        
    m_attachments = m_attachments.strip() + ']'
    display_message = m_datetime + ' ' + message.name + ':' + m_text + ' ' + m_attachments
    
    return display_message

In [17]:
def format_image_name(message):
    message_datetime_utc = message.created_at                                                     
    message_datetime_local = message_datetime_utc.astimezone(to_zone)
    m_datetime = message_datetime_local.strftime('%Y%m%d_%H%M%S')
    
    image_name = m_datetime + '_' + message.name + '.jpg'
    
    return image_name

### Group Summary

In [18]:
message_count = 0
attachment_count = 0
multiple_count = 0
saved_image_count = 0

save_images_flag = True

userCount = collections.Counter()
attachmentTypeCount = collections.Counter()

try:
    for message in theGroup.messages.list_all():
        message_count += 1
        userCount[message.name] += 1
        if message.attachments:
            attachment_count += 1
            if len(message.attachments) > 1:
                multiple_count += 1
            for attachment in message.attachments:
                #print('Attachment: ' + attachment.type)
                attachmentTypeCount[attachment.type] += 1
                #save images
                if attachment.type == 'image' and save_images_flag:
                    saved_image_count += 1
                    image_name = format_image_name(message)
                    image_data = client.images.download(attachment)
                    with open(directory+'/'+image_name, 'wb') as w:
                        w.write(image_data)

        #testing
        #if message_count > 100:
        #    break 

except:
    print('problem - message: ' + str(message_count))
        
# summary results 
summary_text = theGroup.name + "\n"

summary_text = summary_text + '-'*40 + "\n"
summary_text = summary_text + 'Updated as of: ' + str(datetime.now().strftime('%Y-%m-%d %H:%M:%S')) + "\n"
summary_text = summary_text + "\n"
summary_text = summary_text + 'Messages in group: ' + str(message_count) + "\n"
summary_text = summary_text + "Posts by user: \n"
for k, v in userCount.items():
    summary_text = summary_text + "    " + k.ljust(20) + '--> ' + str(v) + "\n"
summary_text = summary_text + "\n"
summary_text = summary_text + "Attachments by type: \n"
for k, v in attachmentTypeCount.items():
    summary_text = summary_text + "    " + k.ljust(20) + '--> ' + str(v) + "\n"
summary_text = summary_text + "\n"
summary_text = summary_text + 'Saved images in folder: ' + str(saved_image_count) + "\n"

summary_text = summary_text + "\n"*3
summary_text = summary_text + 'Done' + "\n"

# Save file
with open(directory+'/Group_Summary.txt', 'w') as w:
    w.write(summary_text)
w.close    

print('Results')
print('-'*40)
print(summary_text)

problem - message: 37900
Results
----------------------------------------
2019 NHL Draft
----------------------------------------
Updated as of: 2019-02-17 17:03:21

Messages in group: 37900
Posts by user: 
    Mike Smith          --> 8213
    Steve               --> 17643
    Brent Kellington    --> 7411
    Rob Kellington      --> 2165
    GroupMe             --> 434
    David               --> 70
    Gerald Barkman      --> 365
    Dave Horrocks       --> 166
    Sean Solbak         --> 128
    Steve McKeigan      --> 37
    Trevor Ranger       --> 1119
    IFTTT via Steve Campbell--> 104
    Steve Campbell      --> 8
    Kelson Smith        --> 12
    Michael Smith       --> 1
    GroupMe Calendar    --> 2
    Hockey Bot          --> 1
    Jonathan Stachniak  --> 17
    Parker Kellington   --> 1
    David Gannon        --> 1
    Terry Miller        --> 2

Attachments by type: 
    image               --> 1862
    mentions            --> 93
    emoji               --> 308
    locati

### BELOW IS WORKING AREA

In [65]:
#Overview of the last message:
message.__dict__

{'data': {'attachments': [{'type': 'image',
    'url': 'https://i.groupme.com/220x275.gif.388c61de86c644a2bf540fe5c912713e'}],
  'avatar_url': 'https://i.groupme.com/1022x1024.jpeg.cd3aa7ca88984d98976750b471a9d35d',
  'created_at': 1550365828,
  'favorited_by': [],
  'group_id': '8234637',
  'id': '155036582874205182',
  'name': 'Steve',
  'sender_id': '2160970',
  'sender_type': 'user',
  'source_guid': '252373D2-41E8-42FF-9763-00CC2E0D9E01',
  'system': False,
  'text': None,
  'user_id': '2160970'},
 'manager': <groupy.api.messages.Messages at 0x10e002f98>,
 'conversation_id': '8234637',
 'created_at': datetime.datetime(2019, 2, 17, 1, 10, 28, tzinfo=datetime.timezone.utc),
 'attachments': [<groupy.api.attachments.Image at 0x10e0915c0>],
 '_likes': <groupy.api.messages.Likes at 0x10e091470>}

In [6]:
#Overview of the group:
theGroup.__dict__

{'data': {'id': '8234637',
  'group_id': '8234637',
  'name': '💩 Misc Crap',
  'phone_number': '+1 6317147436',
  'type': 'private',
  'description': '',
  'image_url': 'https://i.groupme.com/638x638.jpeg.9b905cb0b87a0131d1f322000a2d0abb',
  'creator_user_id': '2160970',
  'created_at': 1399510869,
  'updated_at': 1550365828,
  'office_mode': False,
  'share_url': 'https://groupme.com/join_group/8234637/E2mtYW',
  'share_qr_code_url': 'https://image.groupme.com/qr/join_group/8234637/E2mtYW/preview',
  'members': [{'user_id': '3302203',
    'nickname': 'Karen Tarvyd',
    'image_url': '',
    'id': '122977328',
    'muted': False,
    'autokicked': False,
    'roles': ['user'],
    'name': 'Karen Tarvyd'},
   {'user_id': '2160970',
    'nickname': 'Steve',
    'image_url': 'https://i.groupme.com/1022x1024.jpeg.cd3aa7ca88984d98976750b471a9d35d',
    'id': '45226384',
    'muted': False,
    'autokicked': False,
    'roles': ['admin', 'owner'],
    'name': 'Steve'},
   {'user_id': '135290

In [7]:
for member in theGroup.members:
    print(member.nickname)

Karen Tarvyd
Steve
Trevor Ranger
Dave Horrocks
Michael Smith
Brent Kellington
Rob Kellington
Maureen Horrocks
Susan K
Fareeza
Jesse Horrocks


In [ ]:
message.__dict__

In [55]:
format_mesage(message)

'2019-02-16 13:26:46 Fareeza:In what direction Rob?  []'

### IMAGES

In [64]:
format_image_name(message)

'20190216_181028_Steve.jpg'

In [66]:
image_data = client.images.download(attachment)

In [67]:
type(image_data)

bytes

In [ ]:
with open('rob.jpg', 'wb') as w:
    w.write(image_data)

In [ ]:
from IPython.display import Image
image = Image(image_data)
image

## Looping

In [ ]:
method1 = theGroup.messages.list_all()
print(type(method1))

In [ ]:
# this is expensive!!!!!
# all_messages = list(theGroup.messages.list().autopage())

In [ ]:
>>> oldest_message_in_page = messages[-1]
>>> page_two = chat_or_group.messages.list_before(oldest_message_in_page.id)
>>> all_messages = list(chat_or_group.messages.list().autopage())

In [ ]:
for message in theGroup.messages.list_all():

## Below is Scrap

In [ ]:
# times are posix 
ts = int("1550349100")
utc = datetime.utcfromtimestamp(ts)

print(utc.strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

# Convert time zone
utc = utc.replace(tzinfo=from_zone)
local = utc.astimezone(to_zone)

In [17]:
# Posting a message 
#newMessage = theGroup.post(text='hi')
newMessage.__dict__

NameError: name 'newMessage' is not defined